In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_absolute_error
import seaborn as sns
import matplotlib.pyplot as plt

import numpy as np
from scipy.sparse import load_npz
from sklearn.metrics import mean_absolute_error

from scipy.sparse import coo_matrix
from scipy.sparse.linalg import svds
from sklearn.metrics import mean_absolute_error


In [2]:
# Load the data
data = pd.read_csv('ml-100k/u.data', sep='\t', header=None,
                   names=['user_id', 'item_id', 'rating', 'timestamp'])

folds = []
n_users = 943
n_items = 1682

#here we load all the folds
for i in range(1, 6):
    train = pd.read_csv('ml-100k/u' + str(i) + '.base', sep='\t', header=None,
                        names=['user_id', 'item_id', 'rating', 'timestamp'])
    test = pd.read_csv('ml-100k/u' + str(i) + '.test', sep='\t', header=None,
                       names=['user_id', 'item_id', 'rating', 'timestamp'])
    ratings = np.zeros((n_users, n_items))

    for row in train.itertuples():
        ratings[row[1]-1, row[2]-1] = row[3]
    tratings = np.zeros((n_users, n_items))

    for row in test.itertuples():
        tratings[row[1]-1, row[2]-1] = row[3]
    folds.append({'train': train, 'test': test, 'ratings': ratings, 'testratings': tratings})


In [16]:
import numpy as np
from scipy.sparse.linalg import svds

# this is the schatten p norm minimization algorithm
def schatten_p_norm_minimization(y, p, lamda, test, max_iter=100):

    # Construct the restriction operator M
    M = train_matrix != 0
    # Initialize the vector x
    x = np.random.randint(1,6,y.shape)
    x_prev = x.copy()
    # Iterate until convergence or max iterations
    for i in range(max_iter):
        # Perform one iteration of the iterative algorithm
        xk = x_prev + y - np.multiply(M , x_prev)
        U, S, V = svds(xk)
        S = np.diag(S)
        S_ = np.zeros_like(S)
        for i in range(S.shape[0]):
            for j in range(S.shape[1]):
            # define threshold for the soft threshold function
                if S[i, j] != 0:
                    t = (lamda/2)*p*(abs(S[i, j])**(p-1))
                else:
                    t = 0  # only defined when singular value != 0

                # apply soft thresholding
                S_[i, j] = np.sign(S[i, j])*max(abs(S[i, j])-t, 0)

        # Uptate x
        x = U@S_@V
        
        x_prev = x.copy()
    # Compute the NMAE on the test set
    test_data = test
    y_true = test_data['rating'].values


    y_pred = x[test_data['user_id'] -
                    1, test_data['item_id'] - 1]
    
    
    errors = np.array(y_true) - np.array(y_pred)


    nmae = np.mean(np.abs(errors)) / 4

    
    return nmae


# Set the hyperparameters
lamda = 0.2
p_values = [0.1, 0.5, 1]
train_matrix = folds[0]['ratings']
test_data = folds[0]['test']



# Iterate over the p values and run the algorithm for each one
for p in p_values:
    test_nmae = schatten_p_norm_minimization(
        train_matrix, p, lamda,test)
    print(f'p={p}: NMAE={test_nmae}')


p=0.1: NMAE=0.1523598597309254
p=0.5: NMAE=0.15151833168993772
p=1: NMAE=0.1513653935211276
